# Personnalisation avec évaluation temporelle sur un groupe de rétention

Le manuel suit en grande partie le manuel standard, avec la modification supplémentaire consistant à conserver 10 % des données "futures" pour chaque utilisateur. Nous établissons ensuite un système d'inférence pour apporter une recommandation et une évaluation externe sur les données retenues.

L'autre différence mineure est que nous prédisons les vues plutôt que les audiences. Nous estimons qu'il est plus intéressant de démontrer la recommandation de films peu populaires (mais hautement personnalisés) que de recommander des films populaires que tout le monde apprécierait (sans besoin de personnalisation).

In [ ]:
import boto3, os
import json
import numpy as np
import pandas as pd
import time
from botocore.exceptions import ClientError
!pip install tqdm
from tqdm import tqdm_notebook
from metrics import mean_reciprocal_rank, ndcg_at_k, precision_at_k

In [ ]:
suffix = str(np.random.uniform())[4:9]

In [ ]:
bucket = "demo-temporal-holdout-"+   suffix        # replace with the name of your S3 bucket
filename = "DEMO-temporal-holdout.csv"

In [ ]:
!aws s3 mb s3://{bucket}

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Télécharger et exploiter les données de référence

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip
data = pd.read_csv('./ml-1m/ratings.dat', sep='::', names=['USER_ID','ITEM_ID','EVENT_VALUE', 'TIMESTAMP'])

In [ ]:
pd.set_option('display.max_rows', 5)

In [ ]:
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
print('unique users %d; unique items %d'%(
    len(data['USER_ID'].unique()), len(data['ITEM_ID'].unique())))
data

### Extraire 10 % des dernières interactions par utilisateur sous forme de tests de rétention.

In [ ]:
ranks = data.groupby('USER_ID').TIMESTAMP.rank(pct=True, method='first')
data = data.join((ranks>0.9).to_frame('holdout'))
holdout = data[data['holdout']].drop('holdout', axis=1)
data = data[~data['holdout']].drop('holdout', axis=1)

In [ ]:
print('unique users %d; unique items %d'%(
    len(data['USER_ID'].unique()), len(data['ITEM_ID'].unique())))
data

### Charger les données

In [ ]:
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

# Créer un schéma

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "DEMO-temporal-schema-"+suffix,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

## Jeux de données et groupes de jeu de données

### Créer un groupe de jeux de données

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-temporal-dataset-group-"+suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

### Créer un jeu de données de type « Interactions »

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = "DEMO-temporal-dataset-"+suffix
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Permissions de compartiments S3 pour un accès personnalisé

### Attacher une politique au compartiment S3

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

### Créer un rôle qui dispose des bonnes autorisations

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
# sometimes need to wait a bit for the role to be created
time.sleep(45)
print(role_arn)

# Créer vos tâches d'importation de jeux de données

Il s'agit du chargement de vos données d'interactions

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-temporal-dataset-import-job-"+suffix,
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

### Attendre que la tâche d'importation de données et l'exécution de la tâche d'importation de données aient le statut ACTIF

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

# Créer une solution

In [ ]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

Il existe de nombreuses recettes pour différents scénarios. Dans cet exemple, nous ne disposons que de données relatives aux interactions, nous choisirons donc l'une des recettes de base.

| Est-ce possible ? | Recette | Description 
|-------- | -------- |:------------
| Y | aws-popularity-count | Calcule la popularité des articles en fonction du nombre d'événements correspondant à cet article dans le jeu de données des interactions utilisateur-article.
| Y | aws-hrnn | Prédit les articles avec lesquels un utilisateur va interagir. Un réseau neuronal hiérarchiquement récurrent qui peut reproduire l'ordre temporel des interactions utilisateur-article.
| N – requiert des métadonnées | aws-hrnn-metadata | Prédit les articles avec lesquels un utilisateur va interagir. HRNN avec des fonctions supplémentaires dérivées des métadonnées contextuelles (métadonnées d'interaction utilisateur-article), des métadonnées de l'utilisateur (jeu de données de l'utilisateur) et des métadonnées de l'article (jeu de données de l'article)
| N – pour les bandits et requiert des métadonnées | aws-hrnn-coldstart | Prédit les articles avec lesquels un utilisateur va interagir. HRNN – métadonnées avec exploration personnalisée des nouveaux articles.
| N – pour les requêtes basées sur les articles | aws-sims | Calcule les articles similaires à un article spécifique sur la base de la co-occurrence de l'article dans l'historique du même utilisateur dans le jeu de données d'interaction utilisateur-article.
| N – pour le reclassement d'une liste courte | aws-personalized-ranking | Reclasse une liste des articles pour un utilisateur. Entraîne sur le jeu de données des interactions utilisateur-article. 


Nous (ou autoML) pouvons exécuter toutes ces recettes de base et choisir le modèle le plus performant à partir des mesures internes. Nous recommandons d'effectuer des comparaisons, en particulier avec la popularité de référence, afin de constater l'amélioration des mesures grâce à la personnalisation. Cependant, nous choisirons dans cette démo une seule technique – aws-hrnn, pour nous concentrer sur les évaluations externes.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

### Créer et attendre votre solution
Il s’agit d’un processus en 2 étapes
1. Créer une solution
2. Créer une version de la solution

In [ ]:
create_solution_response = personalize.create_solution(
    name = "DEMO-temporal-solution-"+suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

### Attendre que la version de la solution ait le statut ACTIVE (ACTIF)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### Obtenir les métriques de la solution

In [ ]:
get_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_metrics_response, indent=2))

# Créer et attendre la campagne

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-temporal-campaign-"+suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,    
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

### Attendre que la campagne ait le statut ACTIF

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

# Évaluer à l'aide de paramètres externes

Vous trouverez une explication des paramètres d'évaluation à l'adresse suivante https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html

Admettons par exemple que nous recommandions quatre articles et que deux d'entre eux soient utiles, $r=[0,1,0,1]$. Dans ce cas, les paramètres sont les suivants :

|Nom	|Exemple	|Explication
|:------|:----------|:----------
|Precision@K	|$\frac{2}{4} = 0.5$	|Total des articles utiles divisé par le total des articles recommandés.
|Classements réciproques moyens (MRR@K)	|${\rm mean}(\frac{1}{2} + \frac{1}{4}) = 0.375$	|Tient compte des effets de position en calculant la moyenne des positions inverses de tous les éléments utiles.
|Gains cumulés actualisés et normalisés (NDCG@K)	|$\frac{\frac{1}{\log(1 + 2)} + \frac{1}{\log(1 + 4)}}{\frac{1}{\log(1 + 1)} + \frac{1}{\log(1 + 2)}} = 0.65$	|Tient compte des effets de position en appliquant des pondérations logarithmiques inverses basées sur les positions des articles utiles, normalisées par les scores les plus élevés possibles des recommandations idoines.
|Précision moyenne (AP@K)	|${\rm mean}(\frac{1}{2} + \frac{2}{4}) = 0.5$	|Précision moyenne@K où K correspond à la position de chaque élément utiles.

Ces données diffèrent des données internes sur deux points :
* Elles sont évaluées à des périodes différentes, ce qui peut impliquer des taux de clics différents. Nous recommandons de toujours garder les évaluations dans les mêmes intervalles de temps pour éviter les décalages temporels.
* Les exemples d'évaluations externes peuvent retenir et considérer plusieurs éléments comme étant la réalité du terrain, tandis que les évaluations internes ne retiennent que le dernier élément de chaque historique d'utilisateur comme étant la réalité du terrain. Il n'existe pas de préférence absolue quant au nombre d'articles à retenir ; nous recommandons de concevoir des méthodes d'évaluation similaires au cas d'utilisation réel.

In [ ]:
relevance = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    rec_items = [int(x['itemId']) for x in rec_response['itemList']]
    relevance.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in relevance]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in relevance]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in relevance]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in relevance]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in relevance]))

### Facultatif : des résultats sensiblement plus satisfaisants après déduplication des historiques d'achats antérieurs

In [ ]:
rel_dedup = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id)
    )
    past_items = data[data.USER_ID == user_id].ITEM_ID.values
    topk = [int(x['itemId']) for x in rec_response['itemList']]
    rec_items = [x for x in topk if x not in past_items]
    if len(rec_items) < 25:
        rec_items.extend([x for x in topk if x not in rec_items])
    rec_items = rec_items[:25]    

    rel_dedup.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_dedup]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_dedup]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_dedup]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_dedup]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_dedup]))

### Essayez de comparer avec la popularité de base comme un recommandeur fictif.

In [ ]:
topk = data.groupby("ITEM_ID").TIMESTAMP.count().sort_values(ascending=False).iloc[:100].index.values

In [ ]:
rel_popular = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    rec_items = topk[:25]
    rel_popular.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_popular]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_popular]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_popular]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_popular]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_popular]))

### Déduplication des historiques des utilisateurs grâce à la popularité de base

In [ ]:
rel_pop_dedup = []
for user_id, true_items in tqdm_notebook(holdout.groupby('USER_ID').ITEM_ID):
    past_items = data[data.USER_ID == user_id].ITEM_ID.values
    rec_items = [x for x in topk if x not in past_items]
    if len(rec_items) < 25:
        rec_items.extend([x for x in topk if x not in rec_items])
    rec_items = rec_items[:25]    
    rel_pop_dedup.append([int(x in true_items.values) for x in rec_items])

In [ ]:
print('mean_reciprocal_rank', np.mean([mean_reciprocal_rank(r) for r in rel_pop_dedup]))
print('precision_at_5', np.mean([precision_at_k(r, 5) for r in rel_pop_dedup]))
print('precision_at_10', np.mean([precision_at_k(r, 10) for r in rel_pop_dedup]))
print('precision_at_25', np.mean([precision_at_k(r, 25) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_5', np.mean([ndcg_at_k(r, 5) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_10', np.mean([ndcg_at_k(r, 10) for r in rel_pop_dedup]))
print('normalized_discounted_cumulative_gain_at_25', np.mean([ndcg_at_k(r, 25) for r in rel_pop_dedup]))

# Nettoyer

In [ ]:
personalize.delete_campaign(campaignArn=campaign_arn)
while len(personalize.list_campaigns(solutionArn=solution_arn)['campaigns']):
    time.sleep(5)

personalize.delete_solution(solutionArn=solution_arn)
while len(personalize.list_solutions(datasetGroupArn=dataset_group_arn)['solutions']):
    time.sleep(5)

for dataset in personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']:
    personalize.delete_dataset(datasetArn=dataset['datasetArn'])
while len(personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']):
    time.sleep(5)

personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

# Si vous utilisez un compartiment personnel, soyez prudent lorsque vous exécutez cette cellule !

In [ ]:
!aws s3 rm s3://{bucket} --recursive